# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
# Setting Keyspace
try:
    session.set_keyspace('udacity')
except:
    print(e)

#### 1) Query description: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
##### _
##### 1. Designing database model in Apache Cassandra we need to think Queries first - in our scenario our query will look like: 
`SELECT artist, song, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4`
##### 2. Our <em>PRIMARY KEY</em> will consist of: 
###### Partition key: `sessionId`
###### Clustering Key `itemInSession`
###### That will allow us filter by these attributes.
##### 3. Our table will contain the following tables: 
- `sessionId INT PK`
- `itemInSession INT PK`
- `artist TEXT` 
- `song TEXT`
- `song_length FLOAT`

In [ ]:
## Defining queries and executing them
query_drop = "DROP TABLE IF EXISTS session_songs"
query_create = "CREATE TABLE IF NOT EXISTS session_songs "
query_create = query_create + "(sessionId INT, itemInSession INT, artist TEXT, song TEXT, song_length FLOAT, PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query_drop)
    session.execute(query_create)
except Exception as e:
    print(e)
    
                    

In [ ]:
# Process of inserting data to table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    total = 0
    for index, line in enumerate(csvreader):
        if index % 1000 == 0 and index > 0:
            print(f'Inserted rows: {index}')
        query_insert = "INSERT INTO session_songs (sessionId, itemInSession, artist, song, song_length)"
        query_insert = query_insert + " VALUES (%s, %s, %s, %s, %s)"
        artist, user_firstname, gender, itemInSession, user_lastname, song_length, level, location, sessionId, song, userId = line
        session.execute(query_insert, (int(sessionId), int(itemInSession), artist, song, float(song_length)))
        total += 1
    print(f"Total rows inserted: {total}")

In [ ]:
## Veryfing our table with appropriate SELECT query and print each row
first_query_select = "SELECT artist, song, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4"
rows = session.execute(first_query_select)

dataframe_data = []
for row in rows:
    print(f"{row.artist} | {row.song} | {row.song_length}")
    row_data = ({
        "artist": row.artist,
        "song": row.song,
        "song_length": row.song_length
    })
    dataframe_data.append(row_data)
query_dataframe = pd.DataFrame(dataframe_data)

In [ ]:
## Presenting results from SELECT query with Pandas dataframe
query_dataframe

#### 2) Query description: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
##### _
##### 1. Designing database model in Apache Cassandra we need to think Queries first - in our scenario our query will look like: 
`SELECT itemInSession, artist, song, user_firstname, user_lastname FROM user_songs WHERE userId = 10 AND sessionId = 182`
##### 2. Our <em>COMPOSITE PRIMARY KEY</em> will consist of: 
###### Partition key: `userId`
###### Partition key: `sessionId`
###### Clustering Key `itemInSession`
###### That will allow us filter by these attributes.
##### 3. Our table will contain the following tables: 
- `userId INT PK`
- `sessionId INT PK`
- `artist TEXT` 
- `song TEXT`
- `user_firstname TEXT`
- `user_lastname TEXT`
- `itemInSession INT PK`

In [ ]:
## Defining queries and executing them
query_drop = "DROP TABLE IF EXISTS user_songs"
query_create = "CREATE TABLE IF NOT EXISTS user_songs "
query_create = query_create + "(userId INT, sessionId INT, artist TEXT, song TEXT, user_firstname TEXT, user_lastname TEXT, itemInSession INT, PRIMARY KEY((userId, sessionId), itemInSession))"
try:
    session.execute(query_drop)
    session.execute(query_create)
except Exception as e:
    print(e)
    

In [ ]:
## Process of inserting data to table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    total = 0
    for index, line in enumerate(csvreader):
        if index % 1000 == 0 and index > 0:
            print(f'Inserted rows: {index}')
        query = "INSERT INTO user_songs (userId, sessionId, artist, song, user_firstname, user_lastname, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist, user_firstname, gender, itemInSession, user_lastname, song_length, level, location, sessionId, song, userId = line
        session.execute(query, (int(userId), int(sessionId), artist, song, user_firstname, user_lastname, int(itemInSession)))
        total += 1
    print(f"Total rows inserted: {total}")

In [ ]:
## Veryfing our table with appropriate SELECT query and print each row
first_query_select = "SELECT itemInSession, artist, song, user_firstname, user_lastname FROM user_songs WHERE userId = 10 AND sessionId = 182"
rows = session.execute(first_query_select)

dataframe_data = []
for row in rows:
    print(f"{row.iteminsession} | {row.artist} | {row.song} | {row.user_firstname} | {row.user_lastname}")
    row_data = ({
        "itemInSession": row.iteminsession,
        "artist": row.artist,
        "song": row.song,
        "user_firstname": row.user_firstname,
        "user_lastname": row.user_lastname
    })
    dataframe_data.append(row_data)
query_dataframe = pd.DataFrame(dataframe_data)

In [ ]:
## Presenting results from SELECT query with Pandas dataframe
query_dataframe

#### 3) Query description: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
##### _
##### 1. Designing database model in Apache Cassandra we need to think Queries first - in our scenario our query will look like: 
`SELECT user_firstname, user_lastname FROM app_history WHERE song = 'All Hands Against His Own'`
##### 2. Our <em>PRIMARY KEY</em> will consist of: 
###### Partition key: `song`
###### Clustering Key `userId`
###### That will allow us filter by these attributes.
##### 3. Our table will contain the following tables: 
- `song INT PK`
- `user_firstname TEXT` 
- `user_lastname TEXT`
- `userId INT PK`


In [ ]:
## Defining queries and executing them
query_drop = "DROP TABLE IF EXISTS app_history"
query_create = "CREATE TABLE IF NOT EXISTS app_history "
query_create = query_create + "(song TEXT, user_firstname TEXT, user_lastname TEXT, userId INT, PRIMARY KEY(song, userId))"
try:
    session.execute(query_drop)
    session.execute(query_create)
except Exception as e:
    print(e)

In [ ]:
## Process of inserting data to table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    total = 0
    for index, line in enumerate(csvreader):
        if index % 1000 == 0 and index > 0:
            print(f'Inserted rows: {index}')
        query = "INSERT INTO app_history (song, user_firstname, user_lastname, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        artist, user_firstname, gender, itemInSession, user_lastname, song_length, level, location, sessionId, song, userId = line
        session.execute(query, (song, user_firstname, user_lastname, int(userId)))
        total += 1
    print(f"Total rows inserted: {total}")

In [ ]:
## Veryfing our table with appropriate SELECT query and print each row
first_query_select = "SELECT user_firstname, user_lastname FROM app_history WHERE song = 'All Hands Against His Own'"
rows = session.execute(first_query_select)

dataframe_data = []
for row in rows:
    print(f"{row.user_firstname} | {row.user_lastname}")
    row_data = ({
        "user_firstname": row.user_firstname,
        "user_lastname": row.user_lastname
    })
    dataframe_data.append(row_data)
query_dataframe = pd.DataFrame(dataframe_data)

In [ ]:
## Presenting results from SELECT query with Pandas dataframe
query_dataframe

## Dropping tables:

In [ ]:
## Drop the tables before closing out the sessions
table_names = ['session_songs', 'user_songs', 'app_history']
for table in table_names:
    query_drop = f"DROP TABLE IF EXISTS {table}"
    try:
        session.execute(query_drop)
        print(f"Table '{table}' has been deleted")
    except Exception as e:
        print(e)


In [ ]:
## Close the session and cluster connection
session.shutdown()
cluster.shutdown()